<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Valid_97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Model


In [ ]:
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
model = ViTForImageClassification.from_pretrained(f"/content/Model/Models-Train-27/checkpoint-3000").to(device)
image_processor = ViTImageProcessor.from_pretrained(f"/content/Model/Models-Train-27/checkpoint-3000")

# Standard Experiment

In [ ]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/Dataset(s)/joint-food-101/test", split="train")

In [ ]:
ds

# 100% Missing Modality Experiment

In [ ]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/Dataset(s)/jointHF-train+test_unseen/test", split="train")

In [ ]:
ds

In [ ]:
# Use _3.png for Filtering out Encoded Images
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_3.png")

In [ ]:
ds = ds.filter(filter_funtion)

In [ ]:
ds

In [ ]:
# for i in range(0,10):
#     sample = ds[i]
#     img = sample["image"]
#     filename = os.path.basename(img.filename)
#     print(f"File Name: {filename}")

# Partial Missing Modality Experiment

In [ ]:
from datasets import load_dataset
from datasets import concatenate_datasets

In [ ]:
# Load the Fused Dataset
ds_f = load_dataset("imagefolder", data_dir="/content/Dataset(s)/fused-ferramenta-val", split="train")

In [ ]:
ds_f

In [ ]:
# Desired Percentage = Total - Missing
import math
desired_percentage = 0.9

In [ ]:
from collections import Counter

class_counts = Counter(ds_f['label'])
print(class_counts)

In [ ]:
selected_indices = {label: [] for label in class_counts.keys()}
print(selected_indices)

In [ ]:
# Iterate through the dataset to select indices for each class
for i, label in enumerate(ds_f["label"]):
    if len(selected_indices[label]) < math.ceil(class_counts[label] * desired_percentage):
        selected_indices[label].append(i)

In [ ]:
# selected_indices

In [ ]:
# Flatten the selected indices list
selected_indices = [idx for indices in selected_indices.values() for idx in indices]

In [ ]:
# selected_indices

In [ ]:
# Filter the dataset to select the desired samples
fused_ds = ds_f.select(selected_indices)

In [ ]:
fused_ds

In [ ]:
# Load the Joint Dataset and Filter out Just the Images
ds_joint = load_dataset("imagefolder", data_dir="/content/Dataset(s)/joint-ferramenta/images-val", split="train")

In [ ]:
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_4.png")

In [ ]:
ds_imgs = ds_joint.filter(filter_funtion)

In [ ]:
ds_imgs

In [ ]:
# for i in range(10,100):
#     sample = ds_imgs[i]
#     img = sample["image"]
#     filename = os.path.basename(img.filename)
#     print(f"File Name: {filename}")

In [ ]:
all_indices = range(len(ds_imgs))
print(all_indices)

In [ ]:
remaining_indices = [idx for idx in all_indices if idx not in selected_indices]

In [ ]:
# remaining_indices

In [ ]:
missing_ds = ds_imgs.select(remaining_indices)

In [ ]:
missing_ds

In [ ]:
# Combine Both Datasets

In [ ]:
ds = concatenate_datasets([fused_ds, missing_ds])

In [ ]:
ds

# Pre-Processing and Final Validation Loop

In [ ]:
labels = ds.features["label"]
labels

In [ ]:
labels.int2str(ds[532]["label"])

In [ ]:
def transform(examples):
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]

  return inputs

In [ ]:
dataset = ds.with_transform(transform)

In [ ]:
dataset

In [ ]:
for item in dataset:
  print(item["pixel_values"].shape)
  print(item["labels"])
  break

In [ ]:
labels = ds.features["label"].names
labels

In [ ]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [ ]:
dataset

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from torch.utils.data import DataLoader

In [ ]:
batch_size = 32

In [ ]:
valid_dataset_loader = DataLoader(dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [ ]:
model = model.to(device)
model.eval()
predictions, labels = [], []
# valid_loss = 0
for batch in valid_dataset_loader:
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device)

    outputs = model(pixel_values=pixel_values, labels=label_ids)

    # loss = outputs.loss
    # valid_loss += loss.item()

    logits = outputs.logits.detach().cpu()

    predictions.extend(logits.argmax(dim=-1).tolist())
    labels.extend(label_ids.tolist())

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='weighted')
recall = recall_score(labels, predictions, average='weighted')
f1 = f1_score(labels, predictions, average='weighted')
AUROC_score = roc_auc_score(labels, predictions)

In [ ]:
print("Accuracy: ", accuracy)
print("precision: ", precision)
print("f1_score: ", f1)
print("recall", recall)
print("AUROC_score: ", AUROC_score)

In [ ]:
print(classification_report(labels, predictions))

In [ ]:
cm = confusion_matrix(labels, predictions)
print(cm)

# Joint Representation Experiment


In [ ]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/Dataset(s)/jointHF-train+test_unseen/test", split="train")

Resolving data files:   0%|          | 0/4000 [00:00<?, ?it/s]

In [ ]:
ds

Dataset({
    features: ['image', 'label'],
    num_rows: 4000
})

In [ ]:
# Use _3.png for Filtering out Encoded Images
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_3.png")

In [ ]:
ds_3 = ds.filter(filter_funtion)

In [ ]:
ds_3

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

In [ ]:
for i in range(0,5):
    sample = ds_3[i]
    img = sample["image"]
    filename = os.path.basename(img.filename)
    print(f"File Name: {filename}")

File Name: 01285.jpg_3.png
File Name: 01358.jpg_3.png
File Name: 01397.jpg_3.png
File Name: 01425.jpg_3.png
File Name: 01567.jpg_3.png


Image Dataset

In [ ]:
# Use _3.png for Filtering out Encoded Images
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_4.png")

In [ ]:
ds_4 = ds.filter(filter_funtion)

In [ ]:
ds_4

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

In [ ]:
for i in range(0,5):
    sample = ds_4[i]
    img = sample["image"]
    filename = os.path.basename(img.filename)
    print(f"File Name: {filename}")

File Name: 01285.jpg_4.png
File Name: 01358.jpg_4.png
File Name: 01397.jpg_4.png
File Name: 01425.jpg_4.png
File Name: 01567.jpg_4.png


Pre-processing

In [ ]:
labels = ds.features["label"]
labels

ClassLabel(names=['0', '1'], id=None)

In [ ]:
labels.int2str(ds[532]["label"])

'0'

In [ ]:
def transform(examples):
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]

  return inputs

In [ ]:
dataset_3 = ds_3.with_transform(transform)
dataset_4 = ds_4.with_transform(transform)

In [ ]:
dataset_3

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

In [ ]:
dataset_4

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

In [ ]:
for item in dataset_4:
  print(item["pixel_values"].shape)
  print(item["labels"])
  break

torch.Size([3, 224, 224])
0


In [ ]:
labels = ds.features["label"].names
labels

['0', '1']

In [ ]:
num_of_classes = len(labels)
print(num_of_classes)

2


In [ ]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from torch.utils.data import DataLoader

In [ ]:
batch_size = 32

In [ ]:
valid_dataset_loader_4 = DataLoader(dataset_4, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [ ]:
print(len(valid_dataset_loader_4))

63


In [ ]:
valid_dataset_loader_3 = DataLoader(dataset_3, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [ ]:
print(len(valid_dataset_loader_3))

63


In [ ]:
num_batches = len(valid_dataset_loader_3)
num_batches_with_logits_3 = int(num_batches * 0.3)  # ___% of batches that you want to include (30 for now)
print("Total Batches: ", num_batches)
print("Missing Modaility Batches: ", num_batches_with_logits_3)

Total Batches:  63
Missing Modaility Batches:  18


In [ ]:
import random

controller = random.sample(range(1, num_batches), num_batches_with_logits_3)
controller = sorted(controller)

In [ ]:
controller, len(controller)

([1, 3, 9, 12, 13, 14, 20, 26, 27, 30, 35, 36, 42, 43, 46, 47, 54, 55], 18)

Validation Loops

In [ ]:
model = model.to(device)
model.eval()
predictions_4, labels_4 = [], []
for batch in valid_dataset_loader_4:
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device)

    outputs = model(pixel_values=pixel_values, labels=label_ids)

    logits = outputs.logits.detach().cpu()
    probabilities = torch.softmax(logits, dim=1)

    predictions_4.extend(probabilities.cpu().numpy())
    labels_4.extend(label_ids.cpu().numpy())

In [ ]:
predictions_3 = []
batch_idx = 0
for batch in valid_dataset_loader_3:
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device)

    if batch_idx in controller:
        print(batch_idx)
        outputs = model(pixel_values=pixel_values, labels=label_ids)
        logits = outputs.logits.detach().cpu()
    else:
        length = len(label_ids)
        logits = torch.zeros((length, num_of_classes), device="cpu")

    probabilities = torch.softmax(logits, dim=1)
    predictions_3.extend(probabilities.cpu().numpy())
    batch_idx = batch_idx + 1

1
3
9
12
13
14
20
26
27
30
35
36
42
43
46
47
54
55


In [ ]:
import numpy as np
predictions_3 = np.array(predictions_3)
predictions_4 = np.array(predictions_4)

In [ ]:
predictions_avg = (predictions_3 + predictions_4) / 2
predictions = np.argmax(predictions_avg, axis=1)

In [ ]:
predictions = np.array(predictions)
labels = np.array(labels_4)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='weighted')
recall = recall_score(labels, predictions, average='weighted')
f1 = f1_score(labels, predictions, average='weighted')
AUROC_score = roc_auc_score(labels, predictions)

In [ ]:
print("Accuracy: ", accuracy)
print("precision: ", precision)
print("f1_score: ", f1)
print("recall", recall)
print("AUROC_score: ", AUROC_score)

Accuracy:  0.739
precision:  0.7423050872968091
f1_score:  0.7403246130320118
recall 0.739
AUROC_score:  0.7272000000000001


In [ ]:
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.80      0.77      0.79      1250
           1       0.64      0.68      0.66       750

    accuracy                           0.74      2000
   macro avg       0.72      0.73      0.72      2000
weighted avg       0.74      0.74      0.74      2000



In [ ]:
cm = confusion_matrix(labels, predictions)
print(cm)

[[968 282]
 [240 510]]
